In [1]:
#执行这个文件必须在python 3.8版本下
import os
import numpy as np
import pickle
import edt
import skimage.io as io
from active_learning_utils import save_obj,load_obj
import torch


/home/wangc/anaconda3/envs/test1/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 导入已计算的距离并排序

In [ ]:
#总样本数num1=比例*len(data_dict_org)
#从negtive抽取等比例的样本放到result_dict
#根据org和negtive的len计算num1
#取uncertainy前num个样本放到dict2
#使用save_obj保存

import random

def select_from_uncertainy(uncertainy_path,data_dict1,data_dict2,num,save_path=None):
    #对于negtive,抽取m%的样本
    def extract_random_num_percent_by_key(num,dictionary):#随机抽取字典中num个样本
        keys = list(dictionary.keys())
        random.shuffle(keys)
        randon_dict={key: dictionary[key] for key in keys}

        num_elements = len(keys)
        num_to_extract = int(num_elements * num)
        top_num_percent_keys = keys[:num_to_extract]
        return {key: randon_dict[key] for key in top_num_percent_keys}
    result_dict=extract_random_num_percent_by_key(num,data_dict2)
    #计算从uncertainy抽取的样本数num3
    num1=int(num*len(data_dict_org))
    num2=len(result_dict)
    num3=num1-num2

    #读取uncertainy_dict
    with open(uncertainy_path, 'rb') as file:
        loaded_data = pickle.load(file)
        uncertainy_dict = loaded_data['uncertainy_dict']
    
    #将uncertainy升序排序
    sorted_dict = dict(sorted(uncertainy_dict.items(), key=lambda item: item[1]))
    sorted_list = list(sorted_dict.keys())

    #抽取前sample_number个样本并放入result_dict
    al_list = sorted_list[:num3]
    for i in al_list:
        # print(i)
        temp = i[:-7]
        result_dict[temp] = data_dict1[temp]

    #使用save_obj函数保存
    if save_path is not None:
        save_obj(result_dict, save_path)
    return num1,num2,num3

In [15]:



data_dict_org=load_obj('/home/wangc/now/NaviAirway/saved_objs/for_128_objs/data_dict_org.pkl')
data_dict_only_negtive=load_obj('/home/wangc/now/NaviAirway/saved_objs/for_128_objs/data_dict_only_negtive')
uncertainy_path = '/home/wangc/now/NaviAirway/saved_var/ae1_uncertainy.pkl'

num=0.3
save_path=f'/home/wangc/now/NaviAirway/saved_objs/for_128_objs/training_info_0209/al_info_{int(num*100)}5'
# select_from_uncertainy(uncertainy_path,data_dict_org,data_dict_only_negtive,num,save_path)
# num1,num2,num3=select_from_uncertainy(uncertainy_path,data_dict_org,data_dict_only_negtive,num,save_path)
num1,num2,num3=select_from_uncertainy(uncertainy_path,data_dict_org,data_dict_only_negtive,num,save_path)

In [23]:
candidates_num,select_num=num2,num3
candidates_num,select_num

(3579, 1425)

In [24]:
from active_learning_utils import load_partial_embeddings,select_from_candidates
# 调用函数
crop_size=['128','256']
file_insert=crop_size[0]
file_path1=f'/home/wangc/now/NaviAirway/saved_var/exact09_{file_insert}_op_embeddings_data.pkl'
file_path2=f'/home/wangc/now/NaviAirway/saved_var/lidc_{file_insert}_op_embeddings_data.pkl'

##提取uncertainy大的candidate_num个样本作为候选集

candidates_names=sorted_list[:candidates_num]
candidates_array, candidates_dict , candidates_list = load_partial_embeddings(file_path1, file_path2, candidates_names)
candidates_tensor=torch.from_numpy(candidates_array)
print(candidates_array.shape, len(candidates_dict), len(candidates_list))





(3579, 1, 256, 4, 16, 16) 3579 3579


In [25]:
device = torch.device('cuda:7' if torch.cuda.is_available() else 'cpu')
select_list =select_from_candidates(select_num,candidates_array, candidates_dict , candidates_list,device)
# print(select_list)
# print(len(select_list))

epoch 0
epoch 1
epoch 2
epoch 3
epoch 4
epoch 5
epoch 6
epoch 7
epoch 8
epoch 9
epoch 10
epoch 11
epoch 12
epoch 13
epoch 14
epoch 15
epoch 16
epoch 17
epoch 18
epoch 19
epoch 20
epoch 21
epoch 22
epoch 23
epoch 24
epoch 25
epoch 26
epoch 27
epoch 28
epoch 29
epoch 30
epoch 31
epoch 32
epoch 33
epoch 34
epoch 35
epoch 36
epoch 37
epoch 38
epoch 39
epoch 40
epoch 41
epoch 42
epoch 43
epoch 44
epoch 45
epoch 46
epoch 47
epoch 48
epoch 49
epoch 50
epoch 51
epoch 52
epoch 53
epoch 54
epoch 55
epoch 56
epoch 57
epoch 58
epoch 59
epoch 60
epoch 61
epoch 62
epoch 63
epoch 64
epoch 65
epoch 66
epoch 67
epoch 68
epoch 69
epoch 70
epoch 71
epoch 72
epoch 73
epoch 74
epoch 75
epoch 76
epoch 77
epoch 78
epoch 79
epoch 80
epoch 81
epoch 82
epoch 83
epoch 84
epoch 85
epoch 86
epoch 87
epoch 88
epoch 89
epoch 90
epoch 91
epoch 92
epoch 93
epoch 94
epoch 95
epoch 96
epoch 97
epoch 98
epoch 99
epoch 100
epoch 101
epoch 102
epoch 103
epoch 104
epoch 105
epoch 106
epoch 107
epoch 108
epoch 109
epoch 110


In [26]:
data_dict_org=load_obj('/home/wangc/now/NaviAirway/saved_objs/for_128_objs/data_dict_org.pkl')
save_path1='/home/wangc/now/NaviAirway/saved_objs/for_128_objs/training_info_0209/ae4_info_203.pkl'
data_dict_org
ae4_data_dict={}
for i in select_list:
    sample_name=i[:18]
    if sample_name[-1]=='n':
        sample_name=sample_name[:-1]
    if sample_name[-1]=='.':
        sample_name=sample_name[:-1]
    # print(sample_name)
    if sample_name in data_dict_org.keys():
        ae4_data_dict[sample_name]=data_dict_org[sample_name]
def extract_random_num_percent_by_key(num,dictionary):#随机抽取字典中num个样本
    keys = list(dictionary.keys())
    random.shuffle(keys)
    randon_dict={key: dictionary[key] for key in keys}

    num_elements = len(keys)
    num_to_extract = int(num_elements * num)
    top_num_percent_keys = keys[:num_to_extract]
    return {key: randon_dict[key] for key in top_num_percent_keys}
data_dict_only_negtive=load_obj('/home/wangc/now/NaviAirway/saved_objs/for_128_objs/data_dict_only_negtive')
data_dict_op=extract_random_num_percent_by_key(num,data_dict_only_negtive)
result_dict={**ae4_data_dict, **data_dict_op}
save_obj(result_dict,save_path1)
print(len(ae4_data_dict),len(select_list),len(data_dict_op),len(result_dict))


1425 1425 3579 5004


In [16]:
data_dict_org.keys()

dict_keys(['EXACT09_CASE02_0', 'EXACT09_CASE02_1', 'EXACT09_CASE02_2', 'EXACT09_CASE02_3', 'EXACT09_CASE02_4', 'EXACT09_CASE02_5', 'EXACT09_CASE02_6', 'EXACT09_CASE02_7', 'EXACT09_CASE02_8', 'EXACT09_CASE02_9', 'EXACT09_CASE02_10', 'EXACT09_CASE02_11', 'EXACT09_CASE02_12', 'EXACT09_CASE02_13', 'EXACT09_CASE02_14', 'EXACT09_CASE02_15', 'EXACT09_CASE02_16', 'EXACT09_CASE02_17', 'EXACT09_CASE02_18', 'EXACT09_CASE02_19', 'EXACT09_CASE02_20', 'EXACT09_CASE02_21', 'EXACT09_CASE02_22', 'EXACT09_CASE02_23', 'EXACT09_CASE02_24', 'EXACT09_CASE02_25', 'EXACT09_CASE02_26', 'EXACT09_CASE02_27', 'EXACT09_CASE02_28', 'EXACT09_CASE02_29', 'EXACT09_CASE02_30', 'EXACT09_CASE02_31', 'EXACT09_CASE02_32', 'EXACT09_CASE02_33', 'EXACT09_CASE02_34', 'EXACT09_CASE02_35', 'EXACT09_CASE02_36', 'EXACT09_CASE02_37', 'EXACT09_CASE02_38', 'EXACT09_CASE02_39', 'EXACT09_CASE02_40', 'EXACT09_CASE02_41', 'EXACT09_CASE02_42', 'EXACT09_CASE02_43', 'EXACT09_CASE02_44', 'EXACT09_CASE02_45', 'EXACT09_CASE02_46', 'EXACT09_CAS

In [ ]:
#中心思想:如果一个样本和已被选中的过于相似，则淘汰

#初筛样本集: uncertainy最高的k个样本(top-k)
#导入 初筛样本集 对应的特征表示r1,r2,r3,...,rk
#导入相似性函数sim
#for i in range(1，k)  注:初值是1而不是0
    #对于每个ri
        #初始 score1=1
        #计算 score1=score1+sim(ri,p) 其中p的初值为None
        #进而socre2=(i-1)/score1,取score2大的样本(如果有相同score2，取下标小的)，记作p，放入select set

In [ ]:
#导入候选集，以及对应的特征表示r1,r2,r3,...,rn
#导入相似性函数sim
#for i in range(1，n)  注:初值是1而不是0
    #对于每个ri
        #初始 score1=1
        #计算 score1=score1+sim(ri,p) 其中p的初值为None
        #进而socre2=(i-1)/score1,取score2大的样本(如果有相同score2，取下标小的)，记作p，放入select_set

In [ ]:
# sample_number=len(data_dict1)-len(data_dict2)
# print(sample_number,len(data_dict2)
sample_number=100
al_list=sorted_list[:sample_number]
for i in al_list:
    temp=i[:-7]
    # print(temp,i)
    print(temp)

# save_obj(data_dict2, "/home/wangc/now/NaviAirway/saved_objs/for_128_objs/training_info/"+saved_name)